In [ ]:
import pandas as pd 
import pickle
import datetime
import re

# Atrial Fibrillation (AF) Patients on Rivaroxaban

In [ ]:
with open('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/first_mention_rivaroxaban.pickle', 'rb') as f:
    rivaroxaban = pickle.load(f)

In [ ]:
len(rivaroxaban)

In [ ]:
df = pd.DataFrame(list(rivaroxaban.items()), columns=['client_idcode', 'first_mention_rivaroxaban']) # This code converts a dictionary (rivaroxaban) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_rivaroxaban'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

In [ ]:
df['first_mention_rivaroxaban'].min()

# Echo Reports

In [ ]:
echos = pd.read_csv('/projects/ROCKET AF/data/af_echos.csv') 

In [ ]:
echos.info()

In [ ]:
echos.shape

In [ ]:
echos.head()

In [ ]:
echos['measname'].unique()

In [ ]:
echos = echos[echos['medicalrecordnumber'].isin(ids)]
echos.shape

In [ ]:
echos['studystartdatetime'].isna().sum()

In [ ]:
echos = echos[['medicalrecordnumber', 'studystartdatetime', 'findingcodetext']]

In [ ]:
echos.rename(columns =  {'medicalrecordnumber': 'client_idcode'}, inplace = True)

In [ ]:
echos.head()

# Identifying non-valvular AF patients

In [ ]:
df = pd.merge(echos, df, on='client_idcode', how='right') 

In [ ]:
df.head()

In [ ]:
df['studystartdatetime'].isna().sum()

In [ ]:
df = df[df['studystartdatetime'].notna()]
df['client_idcode'].nunique()

In [ ]:
1509+1076

In [ ]:
df['studystartdatetime'] = pd.to_datetime(df['studystartdatetime'], utc=True).dt.date 

In [ ]:
df.head()

In [ ]:
%%time 
df['difference'] = pd.to_datetime(df['first_mention_rivaroxaban']) - pd.to_datetime(df['studystartdatetime'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df = df.loc[df['in_window'] == True]
df['client_idcode'].nunique()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

## Cleaning text data

In [ ]:
df.loc[0,'findingcodetext']

In [ ]:
df.info()

In [ ]:
def clean_text(text):
    
    if isinstance(text,str):
        
        # Convert to lowercase
        text = text.lower()
   
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
   
        # Remove extra spaces
        text = ' '.join(text.split())
   
    return text

In [ ]:
df['findingcodetext'] =  df['findingcodetext'].apply(clean_text)

In [ ]:
df.loc[0,'findingcodetext']

### Severe mitral stenosis

In [ ]:
df1 = df[df['findingcodetext'].str.contains('severe mitral stenosis', regex=True, na=False)]
df1.shape

In [ ]:
df1['client_idcode'].nunique()

In [ ]:
df1.head()

In [ ]:
df1.loc[920,'findingcodetext']

In [ ]:
ids_1 = set(df1['client_idcode'])
len(ids_1)

### Severe mitral regurgitation

In [ ]:
df2 = df[df['findingcodetext'].str.contains('severe mitral regurgitation', regex=True, na=False)]
df2.shape

In [ ]:
df2['client_idcode'].nunique()

In [ ]:
df2.head()

In [ ]:
df2.loc[2888,'findingcodetext']

In [ ]:
ids_2 = set(df2['client_idcode'])
len(ids_2)

### Moderate mitral stenosis

In [ ]:
df3 = df[df['findingcodetext'].str.contains('there is moderate mitral stenosis', regex=True, na=False)]
df3.shape

In [ ]:
df3['client_idcode'].nunique()

In [ ]:
df3.head()

In [ ]:
df3.loc[35747,'findingcodetext']

In [ ]:
ids_3 = set(df3['client_idcode'])
len(ids_3)

### Moderate mitral regurgiation

In [ ]:
df4 = df[df['findingcodetext'].str.contains('there is moderate mitral regurgitation', regex=True, na=False)]
df4.shape

In [ ]:
df4['client_idcode'].nunique()

In [ ]:
df4.head()

In [ ]:
df4.loc[1020,'findingcodetext']

In [ ]:
ids_4 = set(df4['client_idcode'])
len(ids_4)

-----------------

In [ ]:
ids = ids_1 | ids_2 | ids_3 | ids_4 
len(ids)

In [ ]:
with open ('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/valvular_af.pickle', 'wb') as f:
    pickle.dump(ids,f)